## 4.4.2. ResNet-50 - budowa modelu i trening

#### Importy i ustawienia środowiska

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from tensorflow_addons.losses import SigmoidFocalCrossEntropy

#### Wczytanie danych

In [ ]:
df = pd.read_csv("./data/processed/images_to_train/resnet/train.csv")

#### Funkcja `augmentation`
Funkcja odpowiedzialna za przeprowadzenie losowego odbicia w poziomie, losowej zmiany jasności, losowej zmiany kontrastu, losowego obrotu o wielokrotność 90 stopni oraz losowego przesunięcia w poziomie i pionie.

In [8]:
random_translation_layer = tf.keras.layers.RandomTranslation(0.1, 0.1)

def augmentation(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    image = tf.image.rot90(image, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
    image = random_translation_layer(image, training=True)
    return image

#### Funkcja `load_image`
Funkcja odpowiedzialna za wczytanie obrazu, jego dekodowanie, skalowanie do rozmiaru 224×224 oraz normalizację do [0, 1]. Dodatkowo, w zależności od etykiety z określonym prawdopodobieństwem, stosuje augmentacje obrazu.

In [9]:
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (224, 224)) / 255.0
    if tf.equal(label, 1):
        image = augmentation(image)
    elif tf.random.uniform(()) < 0.1:
        image = augmentation(image)
    return image, label

#### Funkcja `make_dataset`
Funkcja odpowiedzialna za utworzenie zbioru, który wczytuje i przetwarza dane, opcjonalnie je miesza, grupuje w batch'e.

In [ ]:
def make_dataset(paths, labels, batch_size=32, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

#### Funkcja `get_custom_class_weights`
Funkcja odpowiedzialna za obliczenie wag klas z uwzględnieniem nierównowagi danych oraz dodatkowego wzmocnienia klasy mniejszościowej przez parametr weight_ratio.

In [ ]:
def get_custom_class_weights(labels, weight_ratio=2.0):
    class_counts = np.bincount(labels)
    total = sum(class_counts)
    weight_for_0 = total / (2.0 * class_counts[0])
    weight_for_1 = total / (2.0 * class_counts[1]) * weight_ratio
    return {0: weight_for_0, 1: weight_for_1}

#### Funkcja `build_model`
Funkcja odpowiedzialna za zbudowanie i skompilowanie modelu z częściowym odmrożeniem wag, dodaniem warstw końcowych oraz konfiguracją optymalizatora, funkcji straty i metryk.

In [ ]:
def build_model():
    base_model = tf.keras.applications.ResNet50(input_shape=(
        224, 224, 3), include_top=False, weights='imagenet', pooling='avg')
    
    base_model.trainable = True
    for layer in base_model.layers[:-30]:
        layer.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss=SigmoidFocalCrossEntropy(),
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC(name='auc_pr', curve='PR'),
            tf.keras.metrics.AUC(name='auc_roc', curve='ROC')
        ]
    )
    return model

#### Funkcja `get_callbacks`
Funkcja odpowiedzialna za utworzenie zestawu callbacków wspomagających trening

In [ ]:
def get_callbacks():
    return [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_auc_pr', mode='max', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_auc_pr', mode='max', factor=0.2, patience=3, min_lr=1e-6)
    ]

#### Funkcja `evaluate_model`
Funkcja odpowiedzialna za ocenę skuteczności modelu na zbiorze walidacyjnym poprzez obliczenie metryk, a także za wygenerowanie i zapis wykresu krzywej Precision–Recall dla danego folda.

In [ ]:
def evaluate_model(model, val_ds, fold, plots_dir, metrics_summary):
    y_true, y_pred_prob = [], []
    for images, labels in val_ds:
        preds = model.predict(images).flatten()
        y_pred_prob.extend(preds)
        y_true.extend(labels.numpy())

    y_true = np.array(y_true)
    y_pred_prob = np.array(y_pred_prob)
    thresholds = np.linspace(0.1, 0.9, 81)
    f1_scores = [f1_score(y_true, (y_pred_prob >= t).astype(int)) for t in thresholds]
    best_threshold = thresholds[np.argmax(f1_scores)]
    y_pred = (y_pred_prob >= best_threshold).astype(int)

    f1 = f1_score(y_true, y_pred)
    auc_pr_value = roc_auc_score(y_true, y_pred_prob)
    auc_roc_value = tf.keras.metrics.AUC(curve='ROC')(y_true, y_pred_prob).numpy()
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)
    auc_val = auc(recall, precision)
    plt.figure()
    plt.plot(recall, precision, label=f"AUC-PR = {auc_val:.2f}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision–Recall Curve")
    plt.legend()
    plt.grid(True)
    plt.savefig(plots_dir / f"pr_curve_fold_{fold+1}.png")
    plt.close()

    metrics_summary.append({
        "fold": fold+1,
        "f1": f1,
        "auc_pr": auc_pr_value,
        "auc_roc": auc_roc_value,
        "sensitivity": sensitivity,
        "specificity": specificity
    })
    return auc_val

#### Funkcja `train_one_fold`
Funkcja odpowiedzialna za przeprowadzenie treningu i ewaluacji modelu dla jednej iteracji walidacji krzyżowej

In [ ]:
def train_one_fold(fold, train_idx, val_idx, df, plots_dir, metrics_summary):
    train_paths = df.iloc[train_idx]['path'].tolist()
    train_labels = df.iloc[train_idx]['label_risk_group'].tolist()
    val_paths = df.iloc[val_idx]['path'].tolist()
    val_labels = df.iloc[val_idx]['label_risk_group'].tolist()

    train_ds = make_dataset(train_paths, train_labels)
    val_ds = make_dataset(val_paths, val_labels, shuffle=False)
    class_weights = get_custom_class_weights(train_labels)
    model = build_model()
    model.fit(train_ds, validation_data=val_ds, epochs=30,
              callbacks=get_callbacks(), class_weight=class_weights)
    return model, evaluate_model(model, val_ds, fold, plots_dir, metrics_summary)

#### Funkcja `run_cross_validation`
Funkcja odpowiedzialna za wykonanie 5-krotnej walidacji krzyżowej z zachowaniem proporcji klas

In [ ]:
def run_cross_validation(df):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    plots_dir = Path("plots"); 
    plots_dir.mkdir(parents=True, exist_ok=True)
    metrics_summary = []
    best_model = None
    best_auc = 0.0

    for fold, (train_idx, val_idx) in enumerate(skf.split(df['path'], df['label_risk_group'])):
        print(f"\n Fold {fold+1}/5")
        model, auc_score = train_one_fold(fold, train_idx, val_idx, df, plots_dir, metrics_summary)
        if auc_score > best_auc:
            best_auc = auc_score
            best_model = model

    return best_model, metrics_summary

#### Funkcja `save_results`
Funkcja odpowiedzialna za zapisanie modelu i miar do plików

In [ ]:
def save_results(model, metrics):
    model.save("resnet50_best_model.h5")
    pd.DataFrame(metrics).to_csv("crossval_metrics.csv", index=False)

#### Wywołanie potrzebnych funkcji

In [ ]:
best_model, metrics_summary = run_cross_validation(df)
save_results(best_model, metrics_summary)